# 2.7 Przewidywanie zakupów za pomocą drzew decyzyjnych
Mamy wyrobione wstępne intuicje odnośnie zbioru danych, który będziemy analizować, więc możemy przejść do ich modelowania. Zanim jednak tego dokonamy, nasz zbiór wymaga pewnego wstępnego przetwarzania, żeby algorytm uczenia drzewa był w stanie wychwycić zawarte w nim regularności.

In [1]:
import pandas as pd
import numpy as np


In [2]:
purchases_df = pd.read_csv("../data/online_shoppers_intention.csv")
purchases_df.sample(6)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
8748,0,0.0,0,0.000000,4,46.750000,0.000000,0.050000,0.000000,0.0,Nov,1,1,3,3,Returning_Visitor,False,False
7870,1,191.4,0,0.000000,13,679.903333,0.000000,0.002564,34.864231,0.0,Nov,3,2,3,2,Returning_Visitor,True,True
11229,6,96.0,6,251.666667,16,261.000000,0.012821,0.029872,0.000000,0.0,Nov,3,2,7,1,Returning_Visitor,True,False
1024,0,0.0,0,0.000000,5,672.500000,0.000000,0.040000,0.000000,0.0,Mar,2,2,1,1,Returning_Visitor,False,False
761,2,44.0,0,0.000000,26,1636.016667,0.007692,0.021795,0.000000,0.0,Mar,2,6,1,1,Returning_Visitor,False,False
11019,0,0.0,0,0.000000,5,115.625000,0.040000,0.088000,0.000000,0.0,Nov,2,2,3,3,Returning_Visitor,False,False
